In [3]:
# !pip install osmnx networkx pandas numpy matplotlib geopy lxml pickle5 folium keplergl

import os
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import matplotlib.pyplot as plt
import folium
from folium import plugins
import pickle
# from keplergl import KeplerGl

from networkx.classes import MultiDiGraph

os.makedirs('map_data', exist_ok=True)

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


# ДЛЯ СТАТЬИ ОПИСАНИЯ ЕСТЬ ТОЛЬКО ДЛЯ 1-5 ПУНКТОВ, ДАЛЬШЕ НЕ ДОДЕЛАНО ПОКА

##### 1. Загрузка и очистка

Загружаем все дороги (ребра типа drive) из локации с помощью карты с open street map. Используем 3 локации - разные по размеру. Пока хз вставим их все или только одну какую то, выделили 3 чтобы если вся Москва не посчиталась, то хотя бы райончик показать. Итого имеем карты Москвы целиком, САО, Замоскворечья. В каждой выбираем наибольшую связную компоненту, остальные узлы удаляем (это чтобы например как в САО не остался островок вообще за МКАДом не соединенный с самим районом). Также выводим инфу о количестве элементов в графах

## АПДЕЙТ: 

для Москвы берем ограничение 17 км радиус от центра

In [9]:
def load_graph(location: str) -> MultiDiGraph:
    print(f"Загрузка: {location}...")

    G = ox.graph_from_place(
        location,
        network_type='drive',
        simplify=True
    )

    print(f"Загружено: {location}")

    G.remove_nodes_from(list(nx.isolates(G)))

    largest = max(nx.strongly_connected_components(G), key=len)
    G = G.subgraph(largest).copy()

    moscow_center = (55.7558, 37.6173)

    nodes_to_remove = []
    for node in G.nodes():
        lat = G.nodes[node]['y']
        lon = G.nodes[node]['x']
        dist = geodesic(moscow_center, (lat, lon)).meters
        if dist > 17000:
            nodes_to_remove.append(node)

    # Удаляем все узлы сразу
    G.remove_nodes_from(nodes_to_remove)

    print(f"После очистки: {G.number_of_nodes()} узлов, {G.number_of_edges()} рёбер")
    return G

In [10]:
# Загрузить все 3 графа через функцию
graphs = {}

# Москва
G_moscow = load_graph("Moscow, Russia")
graphs['moscow'] = G_moscow

# Крупный район(САО)
G_district_large = load_graph("Северный административный округ, Москва")
graphs['district_large'] = G_district_large

# Малый район(Замоскворечье)
G_district_small = load_graph("Замоскворечье, Москва")
graphs['district_small'] = G_district_small

Загрузка: Moscow, Russia...


KeyboardInterrupt: 

In [ ]:
Gs = {
	'moscow': G_moscow,
	'sao': G_district_large,
	'zmr': G_district_small
}

NameError: name 'G_moscow' is not defined

##### 2. Функциональные веса

Присваиваем макс скорость каждому типу дороги (тип известен из загруженной карты из osmnx). Неизвестные помечаем как среднее примерно 40 км/ч. 

ВАЖНО: метрика travel_time используется для оценки среднего времени прохождения ребра - берем длину дороги (в км) и делим на 70% от макс скорости, умножаем на 60, получаем минуты. Далее эта метрика будет оценивать в целом важность дороги

In [ ]:
speeds = {
    'motorway': 100,
    'trunk': 90,
    'primary': 70,
    'secondary': 60,
    'tertiary': 50,
    'unclassified': 40,
    'residential': 30,
    'living_street': 20,
}
for name, G in Gs.items():
	for u, v, k, data in G.edges(keys=True, data=True):
			highway = data.get('highway', 'unclassified')
			if isinstance(highway, list):
					highway = highway[0]

			# реальная скорость 70% от максимальной
			speed_kph = speeds.get(highway, 40) * 0.7

			length_km = data.get('length', 100) / 1000

			# в минутах
			travel_time = (length_km / speed_kph) * 60

			data['travel_time'] = travel_time
			data['speed_kph'] = speed_kph

##### 3. Набор O-D (Отправление-Назначение) пар

Важный момент снова. Выбираем несколько (конкретно смотри в коде сколько для каждого из трех возможных графов) точек в "центре" (условном центре, в зависимости от графа снова: для Москвы от центра города на 8 км радиус это примерно ТТК, для САО чуть больше 8 км берем, для Замоскворечья поменьше, в итоге чтобы в этом условном центральном районе было намного меньше точек чем на "периферии"), и несколько точек на "периферии". Далее создаем 5000(может поменяю, лучше проверь что написано в коде) пар между этими точками, чтобы каждая пара была "центр"-"периферия". Таким образом мы будем смотреть на важность дорог, примерно моделируя утренний час пик когда все едут в центр

In [ ]:
moscow_center = (55.7558, 37.6173)

for name, G in Gs.items():
	print(f"{name}:")

	if name == 'moscow':
		center_radius = 8000
	elif name == 'sao':
		center_radius = 10000
	elif name == 'zmr':
		center_radius = 2000

	center_nodes = []
	periphery_nodes = []

	for node in G.nodes():
			lat = G.nodes[node]['y']
			lon = G.nodes[node]['x']
			dist = geodesic(moscow_center, (lat, lon)).meters

			if dist < center_radius:
					center_nodes.append(node)
			else:
					periphery_nodes.append(node)

	print(f"Узлов в центральном районе: {len(center_nodes)}")
	print(f"Узлов на периферии: {len(periphery_nodes)}")

	np.random.seed(1984)
	n_center = min(10, len(center_nodes))
	n_periphery = min(5000, len(periphery_nodes))

	center_sample = np.random.choice(center_nodes, n_center, replace=False)
	periphery_sample = np.random.choice(periphery_nodes, n_periphery, replace=False)

	print(f"Выбрано для O-D пар:")
	print(f"    В центральном районе: {n_center}")
	print(f"    На периферии: {n_periphery}")

	print(f"Создание O-D пар...")

	od_pairs = []
	for _ in range(5000):
			origin = np.random.choice(center_sample)
			destination = np.random.choice(periphery_sample)
			if origin != destination:
					od_pairs.append({
							'node_id_origin': origin,
							'node_id_dest': destination
					})

	df_od = pd.DataFrame(od_pairs)
	print(f"Создано {len(df_od)} O-D пар")
	df_od.to_csv(f"./map_data/{name}_od_pairs.csv")
	print(f"Результат сохранён в ./map_data/{name}_od_pairs.csv")

	print()

sao:
Узлов в центральном районе: 655
Узлов на периферии: 895
Выбрано для O-D пар:
    В центральном районе: 10
    На периферии: 895
Создание O-D пар...
Создано 5000 O-D пар
Результат сохранён в ./map_data/sao_od_pairs.csv

sao:
Узлов в центральном районе: 655
Узлов на периферии: 895
Выбрано для O-D пар:
    В центральном районе: 10
    На периферии: 895
Создание O-D пар...
Создано 5000 O-D пар
Результат сохранён в ./map_data/sao_od_pairs.csv

sao:
Узлов в центральном районе: 655
Узлов на периферии: 895
Выбрано для O-D пар:
    В центральном районе: 10
    На периферии: 895
Создание O-D пар...
Создано 5000 O-D пар
Результат сохранён в ./map_data/sao_od_pairs.csv



##### 4. Сохранение графа


Тут важного нет, иди к 5 пункту

In [ ]:
# заюзал pickle, тк graphml плохо сериализует
for name, G in Gs.items():
	filename = f'./map_data/{name}.pkl'
	print(f"Сохранение {name} в {filename}   ...")
	with open(filename, 'wb') as f:
			pickle.dump(G, f)
	print(f"Сохранено {filename}")

Сохранение moscow в ./map_data/moscow.pkl   ...
Сохранено ./map_data/moscow.pkl
Сохранение sao в ./map_data/sao.pkl   ...
Сохранено ./map_data/sao.pkl
Сохранение zmr в ./map_data/zmr.pkl   ...
Сохранено ./map_data/zmr.pkl


In [ ]:
print("Проверка чтения...")
for name, G in Gs.items():
	print(f"{name:}")
	filename = f'./map_data/{name}.pkl'
	with open(filename, 'rb') as f:
			G_check = pickle.load(f)
	print(f"Успешно прочитано: {G_check.number_of_nodes()} узлов, {G_check.number_of_edges()} рёбер")
	assert G_check.number_of_nodes() == G.number_of_nodes()
	assert G_check.number_of_edges() == G.number_of_edges()
	u, v, k, data = list(G_check.edges(keys=True, data=True))[0]
	print(data)
	print()

print(f"Проверка пройдена!")

Проверка чтения...
moscow
Успешно прочитано: 15629 узлов, 32038 рёбер
{'osmid': [31240736, 17151048, 1272911100], 'highway': 'trunk', 'lanes': '5', 'maxspeed': '100', 'name': 'МКАД, 63-й километр', 'oneway': True, 'reversed': False, 'length': np.float64(182.92857286062113), 'bridge': 'yes', 'geometry': <LINESTRING (37.372 55.789, 37.373 55.789, 37.373 55.79, 37.374 55.79)>, 'travel_time': np.float64(0.17421768843868682), 'speed_kph': 62.99999999999999}

sao
Успешно прочитано: 1550 узлов, 3257 рёбер
{'osmid': 230027853, 'highway': 'tertiary', 'lanes': '6', 'maxspeed': 'RU:urban', 'name': 'Дмитровское шоссе', 'oneway': False, 'reversed': False, 'length': np.float64(38.561263922511536), 'travel_time': np.float64(0.06610502386716263), 'speed_kph': 35.0}

zmr
Успешно прочитано: 175 узлов, 366 рёбер
{'osmid': 157320219, 'highway': 'unclassified', 'lanes': '2', 'maxspeed': 'RU:urban', 'name': '3-й Люсиновский переулок', 'oneway': False, 'reversed': True, 'length': np.float64(243.6692548553865

##### 5. Базовый анализ

Базовые статистики для каждого графа, хз надо ли, потому что потом будет в самом алгоритме это учитываться, но если найдете куда пристроить эти цифры - кайф. Если что LCC - доля узлов, которые состоят в наибольшей связной части всего графа

In [ ]:
for name, G in Gs.items():
	print(f"{name}:")
	# LCC fraction
	lcc_nodes = max(nx.weakly_connected_components(G), key=len)
	lcc_fraction = len(lcc_nodes) / G.number_of_nodes()

	# Плотность
	density = nx.density(G.to_undirected())

	# Средняя степень
	degrees = [G.degree(n) for n in G.nodes()]
	avg_degree = np.mean(degrees)

	# travel_time по всем рёбрам
	travel_times = []
	for u, v, k, data in G.edges(keys=True, data=True):
			if 'travel_time' in data:
					travel_times.append(float(data['travel_time']))

	avg_travel_time = np.mean(travel_times)
	max_travel_time = np.max(travel_times)
	min_travel_time = np.min(travel_times)
	median_travel_time = np.median(travel_times)

	stats_output = f"""Узлов: {G.number_of_nodes():,}
	Рёбер: {G.number_of_edges():,}
	LCC fraction: {lcc_fraction:.4f}
	Плотность: {density:.6f}
	Средняя степень: {avg_degree:.2f}
	Travel_time среднее: {avg_travel_time:.2f} мин
	Travel_time макс: {max_travel_time:.2f} мин
	Travel_time медиана: {median_travel_time:.2f} мин
	O-D пар: {len(df_od)}"""

	# Сохрани в файл
	with open(f'./map_data/{name}_base_stats.txt', 'w') as f:
			f.write(stats_output)

	print(stats_output)

	print()

moscow:
Узлов: 15,629
	Рёбер: 32,038
	LCC fraction: 0.9999
	Плотность: 0.000195
	Средняя степень: 4.10
	Travel_time среднее: 0.42 мин
	Travel_time макс: 8.49 мин
	Travel_time медиана: 0.25 мин
	O-D пар: 5000

sao:
Узлов: 1,550
	Рёбер: 3,257
	LCC fraction: 1.0000
	Плотность: 0.001923
	Средняя степень: 4.20
	Travel_time среднее: 0.46 мин
	Travel_time макс: 4.26 мин
	Travel_time медиана: 0.31 мин
	O-D пар: 5000

zmr:
Узлов: 175
	Рёбер: 366
	LCC fraction: 1.0000
	Плотность: 0.016355
	Средняя степень: 4.18
	Travel_time среднее: 0.35 мин
	Travel_time макс: 1.61 мин
	Travel_time медиана: 0.27 мин
	O-D пар: 5000



## Далее смотри файл new_algo_research

##### 6. Визуализация

In [4]:
map_data_folder = './map_data'
Gs = {
	'moscow': pickle.load(open(f"{map_data_folder}/moscow.pkl", 'rb')),
	'sao': pickle.load(open(f"{map_data_folder}/sao.pkl", 'rb')),
	'zmr': pickle.load(open(f"{map_data_folder}/zmr.pkl", 'rb'))
}

In [7]:
# тут вайбкод!

# Центр Москвы
moscow_center = (55.7558, 37.6173)

for name, G in Gs.items():
	print(f'{name}:')

	# Создай карту
	m = folium.Map(
			location=moscow_center,
			zoom_start=11,
			tiles='OpenStreetMap'
	)

	# Добавь слой со всеми узлами сети (выборка для скорости)
	print("\n📍 Добавляю узлы сети...")
	sample_nodes = list(G.nodes())[:min(1000, G.number_of_nodes())]

	for node in sample_nodes:
			lat = G.nodes[node]['y']
			lon = G.nodes[node]['x']

			folium.CircleMarker(
					location=(lat, lon),
					radius=2,
					color='gray',
					fill=True,
					fillOpacity=0.3,
					weight=0.5,
					popup=f"Node {node}"
			).add_to(m)

	print(f"   ✓ Добавлено {len(sample_nodes)} узлов (выборка)")

	# Добавь O-D пары
	print("📌 Добавляю O-D пары...")
	df_od = pd.read_csv(f'./map_data/{name}_od_pairs.csv')
	sample_od = df_od.sample(min(500, len(df_od)), random_state=42)

	for idx, row in sample_od.iterrows():
			o_node = row['node_id_origin']
			d_node = row['node_id_dest']

			if o_node in G.nodes() and d_node in G.nodes():
					o_lat, o_lon = G.nodes[o_node]['y'], G.nodes[o_node]['x']
					d_lat, d_lon = G.nodes[d_node]['y'], G.nodes[d_node]['x']

					# Origin (зелёный)
					folium.CircleMarker(
							location=(o_lat, o_lon),
							radius=5,
							color='green',
							fill=True,
							fillOpacity=0.7,
							weight=2,
							popup=f"Origin {o_node}"
					).add_to(m)

					# Destination (синий)
					folium.CircleMarker(
							location=(d_lat, d_lon),
							radius=5,
							color='blue',
							fill=True,
							fillOpacity=0.7,
							weight=2,
							popup=f"Dest {d_node}"
					).add_to(m)

					# Линия между ними (красная)
					folium.PolyLine(
							locations=[(o_lat, o_lon), (d_lat, d_lon)],
							color='red',
							weight=1,
							opacity=0.5
					).add_to(m)

	print(f"   ✓ Добавлено {len(sample_od)} O-D маршрутов")

	# Добавь легенду
	legend_html = '''
	<div style="position: fixed;
			bottom: 50px; right: 50px; width: 200px; height: 150px;
			background-color: white; border:2px solid grey; z-index:9999;
			font-size:14px; padding: 10px">

	<p><b>Легенда</b></p>
	<p><i class="fa fa-circle" style="color:gray"></i> Узлы сети</p>
	<p><i class="fa fa-circle" style="color:green"></i> Origin (центр)</p>
	<p><i class="fa fa-circle" style="color:blue"></i> Destination (периферия)</p>
	<p style="color:red">━ O-D маршруты</p>
	</div>
	'''
	m.get_root().html.add_child(folium.Element(legend_html))

	# Сохрани
	map_file = f'map_data/{name}_network_od_pairs.html'
	m.save(map_file)
	print(f"\n✓ Карта сохранена: {map_file}")
	print(f"   Открой в браузере для просмотра")

	# Статистика карты
	print(f"\n📊 Статистика карты:")
	print(f"   - Узлов на карте: {len(sample_nodes)}")
	print(f"   - O-D пар на карте: {len(sample_od)}")
	print(f"   - Всего O-D пар в данных: {len(df_od)}")
	print(f"   - Зум: 11 (хороший для всей Москвы)")
	print()
	print()

moscow:

📍 Добавляю узлы сети...
   ✓ Добавлено 1000 узлов (выборка)
📌 Добавляю O-D пары...
   ✓ Добавлено 500 O-D маршрутов

✓ Карта сохранена: map_data/moscow_network_od_pairs.html
   Открой в браузере для просмотра

📊 Статистика карты:
   - Узлов на карте: 1000
   - O-D пар на карте: 500
   - Всего O-D пар в данных: 5000
   - Зум: 11 (хороший для всей Москвы)


sao:

📍 Добавляю узлы сети...
   ✓ Добавлено 1000 узлов (выборка)
📌 Добавляю O-D пары...
   ✓ Добавлено 500 O-D маршрутов

✓ Карта сохранена: map_data/sao_network_od_pairs.html
   Открой в браузере для просмотра

📊 Статистика карты:
   - Узлов на карте: 1000
   - O-D пар на карте: 500
   - Всего O-D пар в данных: 5000
   - Зум: 11 (хороший для всей Москвы)


zmr:

📍 Добавляю узлы сети...
   ✓ Добавлено 175 узлов (выборка)
📌 Добавляю O-D пары...
   ✓ Добавлено 500 O-D маршрутов

✓ Карта сохранена: map_data/zmr_network_od_pairs.html
   Открой в браузере для просмотра

📊 Статистика карты:
   - Узлов на карте: 175
   - O-D пар на

In [ ]:
# У меня на винде kepler не ставится, не тестил

od_map_data = []
for idx, row in df_od.sample(1000).iterrows():
    o_node = row['node_id_origin']
    d_node = row['node_id_dest']

    if o_node in G.nodes() and d_node in G.nodes():
        od_map_data.append({
            'latitude': G.nodes[o_node]['y'],
            'longitude': G.nodes[o_node]['x'],
            'type': 'origin'
        })
        od_map_data.append({
            'latitude': G.nodes[d_node]['y'],
            'longitude': G.nodes[d_node]['x'],
            'type': 'destination'
        })

df_kepler = pd.DataFrame(od_map_data)

# map_kepler = KeplerGl(height=400)
# map_kepler.add_data(data=df_kepler, name='od_pairs')
# map_kepler.save_to_html(file_name='map_data/moscow_kepler.html')